In [1]:
import xarray as xr
P = xr.open_dataset("../../CAMELS_SW/input_data/P_Camels_SW.nc")
display(P)
SWE= xr.open_dataset("../../CAMELS_SW/input_data/SWE_SW.nc")
display(SWE)
target_periods = ['01/01-30/09','01/02-30/09','01/03-30/09','01/04-30/09','01/05-30/09','01/06-30/09','01/07-30/09','01/08-30/09','01/09-30/09']

<xarray.Dataset>
Dimensions:        (lle: 3, nday: 21915, station: 50)
Coordinates:
  * nday           (nday) datetime64[ns] 1961-01-01 1961-01-02 ... 2020-12-31
  * station        (station) object '1069' '1083' '1123' ... '751' '855' '97'
  * lle            (lle) object 'lon' 'lat' 'elev'
Data variables:
    precipitation  (station, nday) float64 ...
    LLE            (station, lle) float64 ...

<xarray.Dataset>
Dimensions:     (station_id: 914, time: 26663)
Coordinates:
  * time        (time) datetime64[ns] 1949-09-01 1949-09-02 ... 2022-08-31
  * station_id  (station_id) int64 10001 10002 10003 10004 ... 11469 11470 11471
    lon         (station_id) float64 ...
    lat         (station_id) float64 ...
Data variables:
    swe         (time, station_id) float64 ...

In [2]:
# Initialize an empty dictionary to store results for each target period
cumulative_precipitation = {}

# Loop over each target period
for period in target_periods:
    start_date, end_date = period.split('-')
    start_day, start_month = map(int, start_date.split('/'))
    end_day, end_month = map(int, end_date.split('/'))
    
    # Filter dataset for the target period across all years
    mask = (P['nday'].dt.month > start_month) | \
           ((P['nday'].dt.month == start_month) & (P['nday'].dt.day >= start_day))
    mask &= (P['nday'].dt.month < end_month) | \
            ((P['nday'].dt.month == end_month) & (P['nday'].dt.day <= end_day))
    
    filtered_ds = P.where(mask, drop=True)
    
    # Group by year and station and calculate cumulative precipitation
    yearly_precip = (
        filtered_ds['precipitation']
        .groupby(filtered_ds['nday'].dt.year)
        .sum(dim='nday')
    )
    
    # Store the result for this period
    cumulative_precipitation[period] = yearly_precip

# Example: Display results for one period (optional)
display(cumulative_precipitation[target_periods[8]])
display(cumulative_precipitation)

<xarray.DataArray 'precipitation' (station: 50, year: 60)>
array([[ 91.69361013,  73.47629609,  59.74352418, ...,  52.01260716,
         80.58593912,  36.76367888],
       [ 60.999244  ,  82.0120251 ,  77.6453691 , ...,  99.70906172,
         96.9100496 , 110.7750142 ],
       [ 34.79850366,  75.14607132,  81.37590749, ...,  64.17996247,
        104.11025941,  60.04163769],
       ...,
       [101.96758138, 112.78490777,  82.0886061 , ..., 165.38250201,
        147.68831361, 103.63341132],
       [ 68.34601516,  62.58057226,  49.9667325 , ...,  35.83661601,
         45.910404  ,  34.8149873 ],
       [ 47.47097508,  68.39649329,  41.33130272, ...,  36.4840076 ,
         95.03809209,  66.9463597 ]])
Coordinates:
  * station  (station) object '1069' '1083' '1123' '1166' ... '751' '855' '97'
  * year     (year) int64 1961 1962 1963 1964 1965 ... 2016 2017 2018 2019 2020

{'01/01-30/09': <xarray.DataArray 'precipitation' (station: 50, year: 60)>
 array([[612.20824356, 692.03242182, 476.3133444 , ..., 414.75540779,
         649.49169314, 679.9926847 ],
        [615.9072559 , 635.5228616 , 582.24582636, ..., 579.64185636,
         717.30024761, 767.8738495 ],
        [478.80730891, 517.22624398, 399.93837017, ..., 399.01118615,
         561.93836397, 537.86729741],
        ...,
        [604.64090517, 987.61310428, 571.51029253, ..., 670.36134199,
         884.87656139, 874.21400622],
        [531.86154906, 494.55018234, 406.19752621, ..., 425.88312855,
         450.75928519, 465.21301513],
        [502.13222106, 474.10652264, 374.33516741, ..., 448.65355856,
         548.70939324, 451.26214556]])
 Coordinates:
   * station  (station) object '1069' '1083' '1123' '1166' ... '751' '855' '97'
   * year     (year) int64 1961 1962 1963 1964 1965 ... 2016 2017 2018 2019 2020,
 '01/02-30/09': <xarray.DataArray 'precipitation' (station: 50, year: 60)>
 array([[556

In [3]:
# Initialize an empty dictionary to store climatological medians for each target period
climatological_medians = {}

# Loop over each target period
for period, data in cumulative_precipitation.items():
    # Calculate the median across the "year" dimension for each station
    median_precipitation = data.median(dim="year")
    
    # Store the result in the climatological_medians dictionary
    climatological_medians[period] = median_precipitation

display(climatological_medians)
# Example: Display the climatological median for one target period
display(climatological_medians['01/04-30/09'])


"""
target_period = '01/04-30/09'  # Beispiel für ein Target-Period

# Hole den klimatologischen Median für das Target-Period und Basin 2299
basin_2007_mean = climatological_medians[target_period].sel(station='2007')
print(f"Klimatologischer Median für Basin 2007 im Zeitraum {target_period}: {basin_2007_mean.values}")
"""

{'01/01-30/09': <xarray.DataArray 'precipitation' (station: 50)>
 array([567.80518637, 656.01146124, 495.6676711 , 662.43060268,
        639.94838017, 557.31740406, 527.91923766, 551.72610889,
        614.32013379, 475.16570737, 749.69569562, 718.90726094,
        685.91217772, 663.82314718, 464.02409205, 509.21049144,
        567.66060654, 482.28890796, 832.18682918, 748.14292822,
        509.07990474, 493.26970825, 497.15023864, 578.11020892,
        583.30597235, 513.64327689, 533.96799604, 719.38209858,
        551.57670471, 454.24855664, 469.47380499, 620.16780043,
        423.74815305, 543.35679811, 485.01514741, 564.19325884,
        472.07765445, 489.27304661, 595.29532532, 626.61186275,
        632.28485113, 618.48986161, 641.9939979 , 493.72455957,
        685.27008641, 510.90056868, 541.64211973, 702.86345818,
        492.97095981, 480.62195138])
 Coordinates:
   * station  (station) object '1069' '1083' '1123' '1166' ... '751' '855' '97',
 '01/02-30/09': <xarray.DataArray '

<xarray.DataArray 'precipitation' (station: 50)>
array([405.86111863, 474.6876633 , 367.21030024, 470.22585257,
       468.65030434, 443.73679914, 401.71476559, 416.15378606,
       435.48334776, 366.5670844 , 486.50103226, 499.204493  ,
       461.22181363, 450.09151793, 355.13706075, 381.80085686,
       414.83660481, 370.67167242, 492.98260752, 532.00785176,
       364.97365004, 378.51276143, 371.605392  , 422.07077695,
       421.02904077, 389.70644117, 403.04282637, 411.89710125,
       404.49106414, 310.71761071, 362.4483436 , 475.07053051,
       332.56907678, 396.81035784, 365.12662962, 399.88218729,
       361.54742166, 364.9739399 , 460.58156191, 441.33786456,
       458.31237998, 418.61308127, 446.89699626, 364.51966404,
       517.1937205 , 359.88889528, 412.92468097, 467.33703115,
       357.4661506 , 383.37343177])
Coordinates:
  * station  (station) object '1069' '1083' '1123' '1166' ... '751' '855' '97'

'\ntarget_period = \'01/04-30/09\'  # Beispiel für ein Target-Period\n\n# Hole den klimatologischen Median für das Target-Period und Basin 2299\nbasin_2007_mean = climatological_medians[target_period].sel(station=\'2007\')\nprint(f"Klimatologischer Median für Basin 2007 im Zeitraum {target_period}: {basin_2007_mean.values}")\n'

In [4]:
import xarray as xr

# Create a list of target periods and corresponding precipitation data
target_periods = list(climatological_medians.keys())
data_arrays = list(climatological_medians.values())

# Combine the data arrays into a single xarray DataArray
p_clim_median = xr.concat(data_arrays, dim="target_period")

# Assign target periods as a new coordinate
p_clim_median = p_clim_median.assign_coords(target_period=("target_period", target_periods))

# Rename the dimensions and data variable for clarity
p_clim_median = p_clim_median.rename({"station": "Station_ID"})
p_clim_median.name = "p_clim_median"

# Convert to an xarray Dataset
P_climatological = p_clim_median.to_dataset()


# Print the structure of the resulting xarray Dataset
print(P_climatological)


<xarray.Dataset>
Dimensions:        (Station_ID: 50, target_period: 9)
Coordinates:
  * Station_ID     (Station_ID) object '1069' '1083' '1123' ... '751' '855' '97'
  * target_period  (target_period) <U11 '01/01-30/09' ... '01/09-30/09'
Data variables:
    p_clim_median  (target_period, Station_ID) float64 567.8 656.0 ... 66.16


In [ ]:
# Extract the p_clim_median values for Station_ID 2355
basin_2355_p_values = P_climatological.sel(Station_ID='2355')

# Convert to a pandas DataFrame for a better display (optional)
basin_2355_p_values_df = basin_2355_p_values.to_dataframe().reset_index()

# Print the results
print(basin_2355_p_values_df)

In [5]:
import xarray as xr

# Define init dates in MM/DD format
init_dates = ['01/01', '01/02', '01/03', '01/04', '01/05', '01/06', '01/07', '01/08', '01/09']

# Initialize a list to store SWE climatological medians for each init date
swe_clim_medians = []

# Loop through each init date and calculate the climatological median
for init_date in init_dates:
    # Parse the init_date into month and day
    day, month = map(int, init_date.split('/'))
    
    # Filter SWE for the specific date (ignoring year)
    swe_for_day = SWE.sel(time=(SWE['time'].dt.month == month) & (SWE['time'].dt.day == day))
    
    # Calculate the climatological median over time for each station
    swe_median_for_day = swe_for_day.median(dim='time')
    
    # Add a new coordinate for the init_date
    swe_median_for_day = swe_median_for_day.assign_coords(init_date=init_date)
    swe_clim_medians.append(swe_median_for_day)

# Combine all init date medians into a single xarray Dataset
SWE_climatological = xr.concat(swe_clim_medians, dim='init_date')

# Rename the data variable for clarity
SWE_climatological = SWE_climatological.rename({'swe': 'swe_clim_median'})

# Print the structure of the resulting xarray Dataset
print(SWE_climatological)



<xarray.Dataset>
Dimensions:          (init_date: 9, station_id: 914)
Coordinates:
    lon              (station_id) float64 14.8 15.62 17.17 ... 17.9 11.7 14.22
    lat              (station_id) float64 56.87 60.62 65.07 ... 60.2 58.98 58.58
  * station_id       (station_id) int64 10001 10002 10003 ... 11469 11470 11471
  * init_date        (init_date) object '01/01' '01/02' ... '01/08' '01/09'
Data variables:
    swe_clim_median  (init_date, station_id) float64 0.0 22.0 72.0 ... 0.0 0.0


In [20]:
import xarray as xr
import os

# Liste der Basins
basin_ids = ['591', '1083', '1169', '1341', '1403', '1537', '1673', '1780']

# Pfad zu den SWE-Dateien
data_dir = "../../CAMELS_SW/output_data/"

# Liste für die kombinierten Daten
all_data = []

# Lade und kombiniere die Daten
for basin_id in basin_ids:
    file_path = os.path.join(data_dir, f"SWE_1979_2022_gapfilled_basin{basin_id}.nc")
    
    try:
        # Lade das Dataset für dieses Basin
        ds = xr.open_dataset(file_path)
        
        # Füge eine neue Koordinate für das Basin hinzu
        ds = ds.assign_coords(basin_id=basin_id)
        
        # Füge das Dataset zur Liste hinzu
        all_data.append(ds)
    except FileNotFoundError:
        print(f"File not found for Basin {basin_id}: {file_path}")
        continue

# Kombiniere alle geladenen Datasets in einem Dataset
if all_data:
    combined_dataset = xr.concat(all_data, dim="basin_id")
else:
    raise ValueError("No data files were loaded.")

# Ergebnis anzeigen
display(combined_dataset)



<xarray.Dataset>
Dimensions:         (basin_id: 8, station_id: 15, time: 26663)
Coordinates:
  * time            (time) datetime64[ns] 1949-09-01 1949-09-02 ... 2022-08-31
  * station_id      (station_id) object '10050' '10090' ... '11350' '11373'
    lat             (basin_id, station_id) float64 66.89 nan nan ... nan nan nan
    lon             (basin_id, station_id) float64 18.02 nan nan ... nan nan nan
    station_name    (basin_id, station_id) object 'Station_1' nan ... nan nan
  * basin_id        (basin_id) object '591' '1083' '1169' ... '1673' '1780'
Data variables:
    SWE             (basin_id, time, station_id) float64 nan nan nan ... nan nan
    flag            (basin_id, time, station_id) float64 0.0 nan nan ... nan nan
    donor_stations  (basin_id, time, station_id) object '' nan nan ... nan nan